In [ ]:
#@title 👇 安装 Waifuc
print("安装 Waifuc 中")
%pip install huggingface_hub git+https://github.com/deepghs/waifuc.git@main#egg=waifuc
print("安装 Waifuc 完成")

In [ ]:
#@title 👇 爬取图片
import os
from waifuc.action import (
    HeadCountAction,
    NoMonochromeAction,
    FilterSimilarAction,
    TaggingAction,
    PersonSplitAction,
    FaceCountAction,
    FirstNSelectAction,
    CCIPAction,
    ModeConvertAction,
    ClassFilterAction,
    RandomFilenameAction,
    AlignMinSizeAction
)
from waifuc.export import SaveExporter
from waifuc.source import DanbooruSource, YandeSource

tag = "nikorashi-ka" #@param {type:"string"}
count = "5" #@param {type:"string"}
path = "/content/waifuc" #@param {type:"string"}
resource = "danbooru" #@param ["danbooru", "yande.re"]

print(f"开始爬取图片, 爬取标签: {tag}, 保存路径: {path}")

if resource == "danbooru":
    source = DanbooruSource([tag])
elif resource == "yande.re":
    source = YandeSource([tag])

source.attach(
    HeadCountAction(1), # only 1 head,
    ModeConvertAction('RGB', 'white'), # 以RGB色彩模式加载图像并将透明背景替换为白色背景
    ClassFilterAction(['illustration', 'bangumi']),  # 丢弃漫画或3D图像
    FilterSimilarAction('all'),  # 丢弃相似或重复的图像
)[:int(count)].export(
    SaveExporter(os.path.join(path, f"waifuc_{tag}"))
)

print(f"爬取图片完成, 保存路径: {path}")

In [ ]:
#@title 👇 上传爬取的数据到 HuggingFace
import os
from huggingface_hub import HfApi


api = HfApi()
repo_id = "" #@param {type:"string"}
repo_type = "dataset" #@param ["model", "dataset", "space"]
upload_path = "/content/waifuc" #@param {type:"string"}
hf_token = "" #@param {type:"string"}


print(f"上传 {upload_path} 到 {repo_id} (类型: {repo_type}) 中")
os.environ["HF_TOKEN"] = hf_token
try:
    api.upload_folder(
        repo_id=repo_id,
        repo_type=repo_type,
        folder_path=upload_path
    )
except:
    api.upload_large_folder(
        repo_id=repo_id,
        repo_type=repo_type,
        folder_path=upload_path
    )
print(f"上传 {upload_path} 到 {repo_id} (类型: {repo_type}) 完成")